#### catatan: 
beberapa data taksonomi memang kosong dari NCBInya, misal data interaksi virus pycv malah terhubung ke genus capsicum bukan spesies capsicum. dan itu memang dari data GloBInya langsung

In [101]:
# # reset import package
# def reloadPackageOwn():
#     from importlib import reload  
#     import os # we use os.path.join, os.path.basename
#     import sys # we use sys.path
#     import glob # we use glob.glob
#     import importlib # we use importlib.import_module

#     import_folder = os.getcwd()
#     sys.path.append(import_folder) # this tells python to look in `import_folder` for imports
#     for src_file in glob.glob(os.path.join(import_folder, '*.py')):
#         name = os.path.basename(src_file)[:-3]
#         importlib.import_module(name)
#         reload(sys.modules[name])
#         importlib.import_module(name)
        
# reloadPackageOwn()

In [102]:
from tqdm import tqdm
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.embedders import FastText,Word2Vec
from pyrdf2vec.walkers import RandomWalker
from pyvis.network import Network
from sklearn.manifold import TSNE
from umap import UMAP
from SPARQLWrapper import SPARQLWrapper
from jcopml.plot import plot_missing_value

import plotly.express as px
import pandas as pd
import numpy as np
import requests
import os
import networkx as nx
import matplotlib.pyplot as plt

from modul.vectorReferenced import get_taxon_vector,cek_ncbi_id_by_wiki_id_via_string
from modul.filterNodeEdge import removeNodeAndEdgeByFilter,removeEdgesNotInNodes
from modul.helper_umum import contains_string_entire_column,contains_string_entire_column_boolean, minmax, std_scale
#from process import cek_bfs, nx_to_pyviz
from modul.grafHelper import _set_networkx_graph, _plot_nx_by_matplotlib
from modul.visualisasiHelper import embeddingPlot,plotly_graph
from modul.embeddingHelper import df_serangga_to_rdf, rdf_KG_to_embeddings, df_to_dictionary_taxon
from modul.custom_degree_centrality import degree_centrality_custom

In [103]:
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'
dict(get_taxon_vector("Pepper yellow leaf curl virus",ncbi_ontology_url,False))['family']

'NCBI:10811_Geminiviridae'

#### Parameter

In [104]:
data=[
    ('begomovirus_contoh_hasil','Pepper yellow leaf curl virus','Aleyrodidae','Bemisia Tabaci'),
    ('1cucu','Cucumber mosaic virus','Aphididae','Myzus persicae'),
    ('2cri','Tomato chlorosis virus','Aleyrodidae','Bemisia Tabaci'),
    ('3wai','Maize chlorotic dwarf virus','Cicadellidae','Graminella nigrifrons'),
    ('4beg','Tomato yellow leaf curl China virus','Aleyrodidae','Bemisia Tabaci'),
    ('5pol','Cereal yellow dwarf virus','Aphididae','Schizaphis graminum'),
    ('6pea','Pea enation mosaic virus 1','Aphididae','Acyrthosiphon pisum'),
    ('7cucur','Cucurbit yellow stunting disorder virus','Aleyrodidae','Bemisia Tabaci'),
    ('8ten','Rice stripe tenuivirus','Delphacidae','Laodelphax striatellus'),
    ('9fiji','Southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('10capchlo','Capsicum chlorosis orthotospovirus','Thripidae','Thrips Palmi'),
    ('11barley','Barley yellow dwarf virus GAV','Aphididae','Sitobion avenae'),
    ('12tospot','Tomato spotted wilt orthotospovirus','Thripidae','Frankliniella occidentalis'),
    ('13svyv','squash vein yellowing virus','Aleyrodidae','Bemisia Tabaci'),
    ('14sbmv','soybean mosaic virus','Aphididae','Aphis glycines'),
    ('15blv','bean leafroll virus','Aphididae','Acyrthosiphon pisum'),
    ('16rgdv','rice gall dwarf virus','Cicadellidae','Recilia dorsalis'), #sedikit
    ('17srbsdv','southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('18tsrv','tomato severe rugose virus','Aleyrodidae','Bemisia tabaci'),
    ('19gbnv','groundnut bud necrosis virus','Thripidae','Thrips palmi'),
    ('20wbnv','Watermelon bud necrosis virus','Thripidae','Thrips palmi'),
    # error dibawah ini
    # ('+13Poty','Potyvirus','Aphididae','Myzus'),
    # ('+11tung','Tungrovirus','Nilaparvata','Nilaparvata'),
]

data_,nama_virus,acuan_,ujian_=data[2] # vektor acuan  #data virus
# link enpoint sparql ncbi_ontology
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'

#### input data

In [105]:
#1
#baca data
df_node=pd.read_csv('dari_praproses/'+data_+'_node.csv',index_col=0) 
df_edge=pd.read_csv('dari_praproses/'+data_+'_edge.csv',index_col=0)

In [106]:
# df_node[df_node['group']=='virus']
# df_node['group'].unique()

contains_string_entire_column(df_node,'67753_Crinivirus').taxon_id.to_list()

['NCBI:52135',
 'NCBI:67754',
 'NCBI:72750',
 'NCBI:81931',
 'NCBI:1712389',
 'NCBI:31713',
 'NCBI:227507',
 'NCBI:558690',
 'NCBI:51330',
 'NCBI:656520',
 'NCBI:404196',
 'NCBI:267970',
 'NCBI:103881',
 'NCBI:642478']

In [107]:
acuan_

'Aleyrodidae'

In [108]:
# pra-proses khusus proses
# hapus serangga yg cuma famili (mengikuti acuan). soalnya klo cuma tampil famili apa gunanya?
filter_genus_sampai_species_null=(
    (df_node.genus.isnull()) &
    (df_node.species.isnull()) &
    (df_node['class']=='NCBI:50557_Insecta')
)
df_node,df_edge = removeNodeAndEdgeByFilter(df_node[filter_genus_sampai_species_null], df_node,df_edge)

removeNodeAndEdgeByFilter
sebelum : 1203 1858
sesudah : 1197 1851


In [109]:
print(len(df_edge))
df_edge.drop_duplicates(inplace=True)
print(len(df_edge))

1851
1727


In [110]:
# pra-proses khusus proses
#isi data kosong. mengisi takson kosong, dengan takson sebelumnya, untuk tambalan
takson=[
    'superkingdom','kingdom','phylum','class','order','family','genus','species'
]

for x,i in enumerate(takson):
    if (i!='superkingdom'): #selain superkingdom update dengan data sebelumnya
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x-1]]+'^'+i
    else: 
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x+1]]+'^'+i

In [111]:
# pra-proses khusus proses
# eksperimen tambahan. bikin fakta tambahan, yaitu relasi virus utama dengan acuan 
# virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
# serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
# print(len(df_edge))
# for i in virus_utama:
#     for j in serangga_acuan:
#         dict = {'source_taxon_id':i,'target_taxon_id':j,'interaction_type':'pathogenOf'}
#         df_edge = pd.concat([pd.DataFrame(dict,index=[0]), df_edge], ignore_index = True)
#         # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
# print(len(df_edge))

#satu virus saja #perlu ini soalnya kalo tida 2cri nol
virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
print(len(df_edge))
for j in serangga_acuan:
    dict_serangga = {'source_taxon_id':virus_utama[0],'target_taxon_id':j,'interaction_type':'pathogenOf'}
    df_edge = pd.concat([pd.DataFrame(dict_serangga,index=[0]), df_edge], ignore_index = True)
    # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
print(len(df_edge))


1727
1730


In [112]:
# Ini harusnya di praproses. tapi belum fix baiknya hapus atau tidak
# hapus yang bukan virus utama, terakhir akurasi 0.85, kalau berkurang hapus saja ini
# bukan_virus_utama=(df_node['group']=="virus") & (df_node.virus_utama!=True)
# df_node,df_edge = removeNodeAndEdgeByFilter(df_node[bukan_virus_utama], df_node,df_edge)

In [113]:
if(len(df_node[df_node['group']=="serangga"])<=2):
    print("cuma dua serangga")

# Konversi graf

In [114]:
#3
#konversi graph 
gnx = _set_networkx_graph(df_node, df_edge)

# # cuma tampilan, visualisasi graf
# _plot_nx_by_matplotlib(gnx)

# visualisasi Graf

In [115]:
plotly_graph(gnx)

# # # cuma tampilan, visualisasi pyviz
# from pyvis.network import Network
# nt = Network('500px', '900px',directed=True,notebook=True)
# # nt.show_buttons(filter_=['physics'])
# nt.toggle_physics(True)

# for i,data in df_node.iterrows():
#     nt.add_node(
#         data['taxon_id'],
#         label=data['taxon_name'],
#         superkingdom=data['superkingdom'],
#         kingdom=data['kingdom'],
#         filum=data['phylum'],
#         kelas=data['class'],
#         ordo=data['order'],
#         famili=data['family'],
#         genus=data['genus'],
#         spesies=data['species'],
#         # group=data['group'],
#         color=data['color'],
#         )
    
# for i,data in df_edge.iterrows():
#     nt.add_edge(
#         data['source_taxon_id'],
#         data['target_taxon_id'],
#         label=data['interaction_type'])

# # nt.show_buttons(filter_=['physics'])
# nt.show("tmp.fig02.html")

In [116]:
# visualisasi embedding serangga

# embedding
# input : df_node, URL
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL)
# proses
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# sama dua2nya # list_of_entities == transformer._entities 

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga

# plotting
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

# Analisis Interaksi

In [117]:
# from modul.custom_degree_centrality import degree_centrality_custom
# import importlib, sys
# importlib.reload(sys.modules['modul.custom_degree_centrality'])

#4 
# Degree Centrality Custom
virus_utama_ids=list(df_node[df_node['virus_utama']==True].taxon_id)
serangga_ids=list(df_node[df_node['group']=="serangga"].taxon_id)
results_dc = degree_centrality_custom(gnx,virus_utama_ids,serangga_ids)
allnodes = gnx.nodes

Eupeodes fumipennis NCBI:414740 | degree: 1 | langsung: 0 | melalui tanaman: 2
Phthiacnemia picta NCBI:1228037 | degree: 1 | langsung: 0 | melalui tanaman: 2
Epitrix fasciata NCBI:226609 | degree: 1 | langsung: 0 | melalui tanaman: 2
Liriomyza NCBI:127403 | degree: 2 | langsung: 0 | melalui tanaman: 3
Manduca quinquemaculatus NCBI:7128 | degree: 1 | langsung: 0 | melalui tanaman: 2
Lon melane NCBI:2839470 | degree: 3 | langsung: 0 | melalui tanaman: 4
Cantharis livida NCBI:346748 | degree: 1 | langsung: 0 | melalui tanaman: 2
Diglyphus begini NCBI:327393 | degree: 1 | langsung: 0 | melalui tanaman: 2
Apanteles gelechiidivoris NCBI:1911542 | degree: 1 | langsung: 0 | melalui tanaman: 2
Bombus vosnesenskii NCBI:207650 | degree: 2 | langsung: 0 | melalui tanaman: 3
Bombus pensylvanicus NCBI:28643 | degree: 1 | langsung: 0 | melalui tanaman: 2
Bombus centralis NCBI:207625 | degree: 1 | langsung: 0 | melalui tanaman: 2
Anthonomus grandis NCBI:7043 | degree: 1 | langsung: 0 | melalui tanaman

In [118]:
urutan=sorted(results_dc.items(), key=lambda item: item[1], reverse=True)
urutan

[('NCBI:7038', 0.0451505016722408),
 ('NCBI:166114', 0.0033444816053511705),
 ('NCBI:88556', 0.0033444816053511705),
 ('NCBI:414740', 0.0),
 ('NCBI:1228037', 0.0),
 ('NCBI:226609', 0.0),
 ('NCBI:127403', 0.0),
 ('NCBI:7128', 0.0),
 ('NCBI:2839470', 0.0),
 ('NCBI:346748', 0.0),
 ('NCBI:327393', 0.0),
 ('NCBI:1911542', 0.0),
 ('NCBI:207650', 0.0),
 ('NCBI:28643', 0.0),
 ('NCBI:207625', 0.0),
 ('NCBI:7043', 0.0),
 ('NCBI:127406', 0.0),
 ('NCBI:132113', 0.0),
 ('NCBI:586671', 0.0),
 ('NCBI:309944', 0.0),
 ('NCBI:13131', 0.0),
 ('NCBI:598349', 0.0),
 ('NCBI:156285', 0.0),
 ('NCBI:7459', 0.0),
 ('NCBI:161013', 0.0),
 ('NCBI:1350418', 0.0),
 ('NCBI:1004210', 0.0),
 ('NCBI:1511177', 0.0),
 ('NCBI:749652', 0.0),
 ('NCBI:254363', 0.0),
 ('NCBI:69820', 0.0),
 ('NCBI:65032', 0.0),
 ('NCBI:7109', 0.0),
 ('NCBI:29058', 0.0),
 ('NCBI:85658', 0.0),
 ('NCBI:60896', 0.0),
 ('NCBI:30195', 0.0),
 ('NCBI:30192', 0.0),
 ('NCBI:2831026', 0.0),
 ('NCBI:2922386', 0.0),
 ('NCBI:203813', 0.0),
 ('NCBI:219539', 0

In [119]:
#visualisasi data
# mengecek BFS Degree tertinggi
for to_search  in contains_string_entire_column(df_node,'67753_Crinivirus').taxon_id.to_list():#[urutan[0][0]]:#["NCBI:7038"]:#"NCBI:33377","NCBI:7036",
    for edge in nx.bfs_edges(gnx.to_undirected(), source=to_search, depth_limit=1):
            s_id, o_id = edge

            s_label = gnx.nodes[s_id]['label'] +' '+s_id
            o_label = gnx.nodes[o_id]['label'] +' '+o_id
            o_grup = gnx.nodes[o_id]['group']

            # skip subjek to_search
            if (o_grup == 'serangga'):
                print(s_label,'-->', o_label)
    print('===============')

Tomato infectious chlorosis virus NCBI:52135 --> Trialeurodes vaporariorum NCBI:88556
Tomato infectious chlorosis virus NCBI:52135 --> Bemisia tabaci NCBI:7038
Tomato infectious chlorosis virus NCBI:52135 --> Bemisia afer NCBI:166114


# Analisis Taksonomi

In [1]:
#5
# Ambil data NCBI
# data acuan
data_acuan=get_taxon_vector(acuan_,ncbi_ontology_url,False)
print(data_acuan)
data_ujian=get_taxon_vector(ujian_,ncbi_ontology_url)
print(data_ujian)

NameError: name 'get_taxon_vector' is not defined

In [121]:
import importlib, sys
importlib.reload(sys.modules['modul.visualisasiHelper'])
from modul.visualisasiHelper import embeddingPlot,plotly_graph

#6 #konversi node networkx ke RDF
# input : df_node, URL, data_acuan
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL, data_acuan)

#7 #embedding
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
# list entity yang akan diembedd. serangga acuan urutan terakhir
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
list_of_entities.append(f"{URL}#SERANGGA_ACUAN")
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# list_of_entities == transformer._entities # True # artinya sama dua2nya

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga



# visualisasi
# input
# embeddings, list_of_entities, dictionary_serangga
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

In [122]:
#8
#euclidean distance

# buat dataframe
data_to_count=pd.DataFrame(embeddings, columns=list(range(0,100)))

# buat kolom label
ent=[data['label'] for index,data in gnx.nodes(data=True) if(data['group']=='serangga')] #jika serangga
ent.append("#SERANGGA_ACUAN")
data_to_count['label']=ent

#buat kolom entity
data_to_count['entity']=[i.replace("http://pyRDF2Vec#","") for i in transformer._entities]

# buat kolom hasil dc
for idx,row in data_to_count.iterrows(): #jika serangga acuan maka DC di isi nilai 1
    data_to_count.loc[idx,['dc_result']] = results_dc[row['entity']] if(row['entity']!="SERANGGA_ACUAN") else 1

#ambil koordinat acuan
acuan=next(data_to_count[data_to_count['label']=='#SERANGGA_ACUAN'].iterrows())[1]
acuan=np.array(tuple(acuan[i] for i in range(0,100)))
acuan

#hitung ED
for idx, row in data_to_count.iterrows():
    temp = np.array(tuple(row[i] for i in range(0,100)))
    data_to_count.loc[idx,['ed_result']] = np.linalg.norm(temp - acuan)

#drop data acuan
data_to_count.drop(data_to_count[data_to_count.label=="#SERANGGA_ACUAN"].index,inplace=True)

In [123]:
#drop kolom embedding
data_to_count.drop(columns=list(range(0,100)), inplace=True)

In [124]:
# simple scaling ed_result
# data_to_count["ed_result_scaled"] = data_to_count["ed_result"] / data_to_count["ed_result"].max()
data_to_count['ed_result_scaled'] = minmax(data_to_count['ed_result'])


# scaling dc 
# # dengan std
# data_to_count['dc_result_scaled'] = data_to_count['dc_result'] - data_to_count['dc_result'].mode()[0]
# # data_to_count['dc_result_scaled'] = data_to_count['dc_result']
# # lagi dengan minmax
# data_to_count["dc_result_scaled"] = data_to_count["dc_result_scaled"] / data_to_count["dc_result_scaled"].max()

# Perform scaling using standard deviation
# data_to_count['dc_result_scaled'] = std_scale(data_to_count['dc_result'])
# data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result_scaled'])

data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result'])


In [125]:
data_to_count

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled
0,Eupeodes fumipennis,NCBI:414740,0.0,3.237501,0.648739,0.0
1,Phthiacnemia picta,NCBI:1228037,0.0,2.381835,0.455628,0.0
2,Epitrix fasciata,NCBI:226609,0.0,3.280691,0.658487,0.0
3,Liriomyza,NCBI:127403,0.0,3.679827,0.748566,0.0
4,Manduca quinquemaculatus,NCBI:7128,0.0,3.698750,0.752836,0.0
...,...,...,...,...,...,...
498,Harpactus tumidus,NCBI:2495052,0.0,3.412074,0.688138,0.0
499,Oedemera podagrariae,NCBI:433055,0.0,3.758547,0.766332,0.0
500,Eurybia,NCBI:124358,0.0,3.714940,0.756490,0.0
501,Listronotus,NCBI:122862,0.0,3.313536,0.665899,0.0


In [126]:
#9
#hitung kombinasi
# for idx, row in data_to_count.iterrows():
#     _dc = row['dc_result_scaled']
#     _ed=( (row['ed_result_scaled']) if row['ed_result_scaled']!=0 else 1)
#     data_to_count.loc[idx,['result']] = _dc/_ed

data_to_count['result'] = (1+data_to_count['dc_result_scaled']) / (1+data_to_count['ed_result_scaled'])

# simple scaling result (final/kombinasi)
data_to_count['result'] = data_to_count['result'] / data_to_count['result'].max()

# Pengujian

In [127]:
# DC
data_to_count=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)
data_to_count[['label','entity','dc_result']]

,label,entity,dc_result
0,Bemisia tabaci,NCBI:7038,0.045151
1,Bemisia afer,NCBI:166114,0.003344
2,Trialeurodes vaporariorum,NCBI:88556,0.003344
3,Eupeodes fumipennis,NCBI:414740,0.000000
4,Napomyza cichorii,NCBI:414333,0.000000
...,...,...,...
498,Sphecodes confertus,NCBI:479827,0.000000
499,Lasioglossum vierecki,NCBI:88499,0.000000
500,Andrena distans,NCBI:446235,0.000000
501,Hoplitis pilosifrons,NCBI:156336,0.000000


In [128]:

# Pengujian dc
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:7038_Bemisia tabaci', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
1.0
hasil  {'spesies': 'NCBI:166114_Bemisia afer', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.75
hasil  {'spesies': 'NCBI:88556_Trialeurodes vaporariorum', 'genus': 'NCBI:88555_Trialeurodes', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5


In [129]:
# ED
data_to_count=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)
data_to_count[['label','entity','ed_result']]

,label,entity,ed_result
0,Trialeurodes vaporariorum,NCBI:88556,0.362967
1,Bemisia afer,NCBI:166114,0.370646
2,Bemisia tabaci,NCBI:7038,0.391895
3,Corizus,NCBI:1511258,1.875827
4,Stictocephala bisonia,NCBI:54647,1.882671
...,...,...,...
498,Sapyga louisi,NCBI:2086623,4.053847
499,Teleas,NCBI:408499,4.215821
500,Abracris flavolineata,NCBI:58570,4.262224
501,Oecanthus pellucens,NCBI:2567088,4.321300


In [130]:
# Pengujian ed
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:88556_Trialeurodes vaporariorum', 'genus': 'NCBI:88555_Trialeurodes', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5
hasil  {'spesies': 'NCBI:166114_Bemisia afer', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.75
hasil  {'spesies': 'NCBI:7038_Bemisia tabaci', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
1.0


In [131]:
# final score kombinasi
data_to_count=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)
data_to_count#[['label','dc_result','ed_result','result']]

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled,result
0,Bemisia tabaci,NCBI:7038,0.045151,0.391895,0.006529,1.000000,1.000000
1,Trialeurodes vaporariorum,NCBI:88556,0.003344,0.362967,0.000000,0.074074,0.540543
2,Bemisia afer,NCBI:166114,0.003344,0.370646,0.001733,0.074074,0.539608
3,Corizus,NCBI:1511258,0.000000,1.875827,0.341430,0.000000,0.375170
4,Stictocephala bisonia,NCBI:54647,0.000000,1.882671,0.342974,0.000000,0.374739
...,...,...,...,...,...,...,...
498,Sapyga louisi,NCBI:2086623,0.000000,4.053847,0.832976,0.000000,0.274561
499,Teleas,NCBI:408499,0.000000,4.215821,0.869531,0.000000,0.269193
500,Abracris flavolineata,NCBI:58570,0.000000,4.262224,0.880004,0.000000,0.267693
501,Oecanthus pellucens,NCBI:2567088,0.000000,4.321300,0.893336,0.000000,0.265808


In [132]:
# Pengujian kombinasi
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:7038_Bemisia tabaci', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
1.0
hasil  {'spesies': 'NCBI:88556_Trialeurodes vaporariorum', 'genus': 'NCBI:88555_Trialeurodes', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5
hasil  {'spesies': 'NCBI:166114_Bemisia afer', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.75


In [133]:
# Pengujian kombinasi
# cek=1
# takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
# urutan=0
# while cek/len(takson)>0:
#     id_hasil=data_to_count.iloc[urutan].entity
#     cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
#     cek_ujian= { k:v for k,v in data_ujian if k in takson }
#     # print(acuan_,'->', data_)
#     # print('ujian ',cek_ujian)
#     print('hasil ', urutan ,cek_hasil)
#     cek=0
#     for i in reversed(takson):
#         cekk=cek_hasil[i]==cek_ujian[i]
#         cek+=cekk
#         # print(i, cekk)
#     print(cek/len(takson))
#     urutan+=1

# Visualisasi Hasil Analisis

In [134]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('dc_result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Degree centrality',
    xaxis_title='Degree',
    yaxis_title='Insect'
)

fig.show()

In [135]:
to_itter=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('ed_result',ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Euclidean distance',
    xaxis_title='Distance',
    yaxis_title='Insect'
)

fig.show()

In [136]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='final score',
    xaxis_title='Score',
    yaxis_title='Insect'
)

fig.show()

In [137]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the color for all bars
bar_color = '#636EFA'

# First plot
to_itter = data_to_count.sort_values('dc_result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('dc_result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = make_subplots(rows=2, cols=2, subplot_titles=('(a) Degree centrality', '(b) Euclidean distance', '(c) Final score', ''), vertical_spacing=0.1, horizontal_spacing=0.15)
fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=1)

# Second plot
to_itter = data_to_count.sort_values('ed_result', ascending=True).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('ed_result', ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=2)

# Third plot
to_itter = data_to_count.sort_values('result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=2, col=1)

# Hide the empty subplot
fig.update_layout(showlegend=False)
fig.update_annotations(visible=False, selector="title.text")

# Update layout for subplot titles
fig.update_layout(
    # title_text="Combined Plots",
    title_font={'size':24},
    title_x=0.5,
    title_y=0.95,
    title_xanchor='center',
    title_yanchor='top'
)

# Rotate y-axis labels
fig.update_yaxes(tickangle=45)

fig.update_layout(height=800, width=1200)
fig.show()


# Dibawah ini tidak termasuk

In [138]:
# #visualisasi data
# # mengecek BFS Degree tertinggi
# for edge in nx.bfs_edges(gnx.to_undirected(), source="NCBI:7038", depth_limit=1):
#         s_id, o_id = edge

#         s_label = gnx.nodes[s_id]['label'] +' '+s_id
#         o_label = gnx.nodes[o_id]['label'] +' '+o_id
#         o_grup = gnx.nodes[o_id]['group']

#         # skip subjek to_search

#         print(s_label,'-->', o_label)
# print('===============')

In [139]:
df_node[df_node['taxon_name'].str.contains('Thrips')]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
348,NCBI:161013,Thrips palmi,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:161013_Thrips palmi,serangga,#b22222,NaN
349,NCBI:1350418,Thrips parvispinus,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:1350418_Thrips parvispinus,serangga,#b22222,NaN
766,NCBI:45057,Thrips,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | genus,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:45057_Thrips^species,serangga,#b22222,NaN
797,NCBI:161014,Thrips tabaci,Animalia | Arthropoda | Insecta | Thysanoptera...,EOL:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 | ...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:161014_Thrips tabaci,serangga,#b22222,NaN


In [140]:
df_edge.interaction_type.unique()

array(['pathogenOf', 'visitFlowersOf', 'hasHost', 'visit', 'pollinates',
       'patogennya'], dtype=object)

In [141]:
ini=df_edge[df_edge['interaction_type'].isin(['visitFlowersOf'])]
ini

,source_taxon_id,target_taxon_id,interaction_type
42,NCBI:414740,NCBI:4081,visitFlowersOf
359,NCBI:77574,NCBI:4081,visitFlowersOf
360,NCBI:598349,NCBI:4081,visitFlowersOf
361,NCBI:156285,NCBI:4081,visitFlowersOf
362,NCBI:7459,NCBI:4081,visitFlowersOf
...,...,...,...
1633,NCBI:1384776,NCBI:4039,visitFlowersOf
1634,NCBI:32404,NCBI:4039,visitFlowersOf
1635,NCBI:926301,NCBI:4039,visitFlowersOf
1636,NCBI:2794775,NCBI:4039,visitFlowersOf


In [142]:
df_node[df_node['taxon_id'].isin(
    ini.source_taxon_id.to_list()+
    ini.target_taxon_id.to_list()
)]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
0,NCBI:4081,Solanum lycopersicum,root | cellular organisms | Eukaryota | Viridi...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33090 ...,null | null | superkingdom | kingdom | phylum ...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4069_Solanales,NCBI:4070_Solanaceae,NCBI:4107_Solanum,NCBI:4081_Solanum lycopersicum,tanaman,#1f922b,NaN
4,NCBI:71251,Chelidonium majus,Plantae | Tracheophyta | Magnoliopsida | Ranun...,EOL_V2:281 | NCBI:58023 | EOL:283 | EOL:4346 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:41768_Ranunculales,NCBI:3465_Papaveraceae,NCBI:71250_Chelidonium <flowering plants>,NCBI:71251_Chelidonium majus,tanaman,#1f922b,NaN
5,NCBI:101015,Fragaria virginiana,Plantae | Tracheophyta | Magnoliopsida | Rosal...,GBIF:6 | NCBI:58023 | NCBI:3398 | NCBI:3744 | ...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:3744_Rosales,NCBI:3745_Rosaceae,NCBI:3746_Fragaria,NCBI:101015_Fragaria virginiana,tanaman,#1f922b,NaN
6,NCBI:57918,Fragaria vesca,Plantae | Tracheophyta | Magnoliopsida | Rosal...,GBIF:6 | NCBI:58023 | NCBI:3398 | NCBI:3744 | ...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:3744_Rosales,NCBI:3745_Rosaceae,NCBI:3746_Fragaria,NCBI:57918_Fragaria vesca,tanaman,#1f922b,NaN
7,NCBI:161934,Beta vulgaris,root | cellular organisms | Eukaryota | Viridi...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33090 ...,null | null | superkingdom | kingdom | phylum ...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:3524_Caryophyllales,NCBI:1804623_Chenopodiaceae,NCBI:3554_Beta,NCBI:161934_Beta vulgaris,tanaman,#1f922b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,NCBI:253735,Anacrabro ocellatus ocellatus,Animalia | Arthropoda | Hexapoda | Insecta | P...,INAT_TAXON:1 | NCBI:6656 | NCBI:6960 | NCBI:50...,kingdom | phylum | subphylum | class | subclas...,species,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:253718_Crabronidae,NCBI:253734_Anacrabro,NCBI:253735_Anacrabro ocellatus,serangga,#b22222,NaN
1156,NCBI:520261,Parancistrocerus fulvipes,Animalia | Arthropoda | Hexapoda | Insecta | P...,INAT_TAXON:1 | NCBI:6656 | NCBI:6960 | NCBI:50...,kingdom | phylum | subphylum | class | subclas...,genus,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7438_Vespidae,NCBI:520261_Parancistrocerus,INAT_TAXON:450990_Parancistrocerusfulvipes,serangga,#b22222,NaN
1167,NCBI:1352479,Epistrophe grossulariae,Animalia | Arthropoda | Hexapoda | Insecta | P...,INAT_TAXON:1 | NCBI:6656 | NCBI:6960 | NCBI:50...,kingdom | phylum | subphylum | class | subclas...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7147_Diptera,NCBI:34680_Syrphidae,NCBI:414810_Epistrophe,NCBI:1352479_Epistrophe grossulariae,serangga,#b22222,NaN
1171,NCBI:7399,Krombeinopyga,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:7399 | ...,kingdom | phylum | class | order | genus,order,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:7399_Hymenoptera,NCBI:7399_Hymenoptera^family,GBIF:4670385_Krombeinopyga,GBIF:4670385_Krombeinopyga^species,serangga,#b22222,NaN


In [143]:
contains_string_entire_column(df_edge,'NCBI:1341303')

,source_taxon_id,target_taxon_id,interaction_type


In [144]:
contains_string_entire_column(df_node,'NCBI:12315')

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
322,NCBI:12315,Tomato aspermy virus,root | Viruses | Riboviria | Orthornavirae | K...,NCBI:1 | NCBI:10239 | NCBI:2559587 | NCBI:2732...,null | superkingdom | null | kingdom | phylum ...,NaN,NCBI:10239_Viruses,NCBI:2732396_Orthornavirae,NCBI:2732406_Kitrinoviricota,NCBI:2732461_Alsuviricetes,NCBI:2732544_Martellivirales,NCBI:39740_Bromoviridae,NCBI:12304_Cucumovirus,NCBI:12315_Tomato aspermy virus,virus,#671f92,NaN
